In [34]:
import pyodbc
import pandas as pd
import config as cfg

In [35]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [36]:
cnxn = pyodbc.connect( 'DRIVER={ODBC Driver 13 for SQL Server};SERVER=' + cfg.mssql['server'] + ';DATABASE=' 
                      + cfg.mssql['database'] + ';UID=' + cfg.mssql['username'] + ';PWD=' + cfg.mssql['password'] )

In [37]:
query = "SELECT * FROM BankView WHERE [State]='NY';"
data = pd.read_sql(query, cnxn, index_col='BankID')
data.head()

,UniqueNum,Name,Address1,Address2,City,State,Zip,Deposit,Lat,Lng,...,MeanPSDistance,PSCount,Take,PDistance,Officers1000,FFLCount,AvgRating,Target,Population,CrimeRate1000
BankID,,,,,,,,,,,,,,,,,,,,,
6509,590864,Ballston Spa National Bank,1973 Western Ave,,Albany,NY,12203,1917000,42.6950,-73.8794,...,7.330062,11,958,0.373815,1.093750,8,NaN,None,98617.0,43.744993
6510,580447,Ballston Spa National Bank,1207 Troy Schenectady Rd,,Latham,NY,12110,3875000,42.7725,-73.8230,...,6.917659,14,1937,0.405223,1.425641,8,NaN,None,NaN,NaN
6511,4660,"Bank of America, National Association",69 State Street,,Albany,NY,12201,435463000,42.6499,-73.7522,...,5.144061,9,217731,0.601389,2.820146,5,4.1,None,98617.0,43.744993
6512,213160,"Bank of America, National Association",1450 Western Avenue,,Albany,NY,12203,97728000,42.6808,-73.8373,...,7.046373,12,48864,0.290482,2.341346,10,3.0,None,98617.0,43.744993
6513,214981,"Bank of America, National Association",1791 Western Avenue,,Albany,NY,12203,62203000,42.6905,-73.8662,...,7.621675,13,31101,0.300045,1.318182,10,5.0,None,98617.0,43.744993


In [38]:
data['CrimeRate1000'].mean()

29.42786864578506

In [39]:
data['Population'].mean()

3629207.8229166665

In [40]:
data.isnull().sum()

UniqueNum               0
Name                    0
Address1                0
Address2                0
City                    0
State                   0
Zip                     0
Deposit                 0
Lat                     9
Lng                     9
ClosestStationID      233
ClosestPSDistance       0
MeanPSDistance          0
PSCount                 0
Take                    0
PDistance               0
Officers1000            0
FFLCount                0
AvgRating            2113
Target               5033
Population           3401
CrimeRate1000        3401
dtype: int64

In [41]:
values = {'CrimeRate1000': data['CrimeRate1000'].mean(), 'Population': data['Population'].mean(), 'AvgRating' : data['AvgRating'].mean()}
data.fillna(value=values, inplace=True)
data.shape

(5033, 22)

## Use Model to Predict banks to Target

In [42]:
try:
    import cPickle as pickle
except ImportError:
    import pickle
    
## Load Model
model_filepath = 'targetbanks_randomforestclassifier.pkl'
in_logreg = open(model_filepath, 'rb')
rfc = pickle.load(in_logreg)
in_logreg.close()

In [43]:
feature_cols = ['ClosestPSDistance', 'Take', 'PDistance', 'Officers1000',
       'FFLCount', 'AvgRating', 'Population', 'CrimeRate1000']
X = data[feature_cols]
data['y_pred'] = rfc.predict(X)

In [44]:
# Total TARGETABLE banks
tgtsum = data[data.y_pred == 1].Name.count();
total = data.Name.count()
print(tgtsum, 'out of', total, ' pct:', round((tgtsum/total * 100)), '%')

1697 out of 5033  pct: 34.0 %


## Show Result

In [45]:
dftarget = data[data.y_pred == 1].copy()
dftarget.drop(['UniqueNum', 
       'Deposit', 'Lat', 'Lng', 'ClosestStationID',
       'MeanPSDistance', 'PSCount', 'Target', 'Population'], axis=1, inplace=True)

## Top 5 banks with highest Take

In [46]:
# Top 5 banks with highest Take
dftop5 = dftarget.sort_values(by=['Take'], ascending=[False]).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred
BankID,,,,,,,,,,,,,,
8936,"JPMorgan Chase Bank, National Association",401 Madison Avenue,,New York,NY,10017,3.320694,174260736,0.367931,5.0,10,1.000000,20.357615,1
8850,"HSBC Bank USA, National Association",452 Fifth Avenue,,New York,NY,10018,2.940012,43650604,0.405999,5.0,10,5.000000,20.357615,1
8934,"JPMorgan Chase Bank, National Association",401 Madison Avenue,,New York,NY,10017,3.320694,33282569,0.367931,5.0,10,1.000000,20.357615,1
8639,"Bank of America, National Association",115 West 42nd Street,,New York,NY,10036,3.113936,22024089,0.388606,5.0,9,3.700000,20.357615,1
8768,"Citibank, National Association",399 Park Avenue,,New York,NY,10022,3.602696,12862500,0.339730,5.0,10,3.665959,20.357615,1


# Use K-Means with RandomForest

In [49]:
feature_cols = [ 'Take', 'PDistance', 'Officers1000', 'FFLCount', 'AvgRating', 'CrimeRate1000']
X = data[feature_cols]

In [50]:
# K-means with 2 clusters is overfit and NOT the best for this data, so try 4,5,6?
km = KMeans(n_clusters=5, random_state=1)
km.fit(X)
data['cluster'] = km.labels_

In [51]:
data_X = X.copy()
data_X['cluster'] = km.labels_
centers = data_X.groupby('cluster').mean()
centers

,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000
cluster,,,,,,
0,7.538388e+04,0.390732,2.254804,7.933267,3.666183,29.443018
1,1.742607e+08,0.367931,5.000000,10.000000,1.000000,20.357615
2,2.669622e+07,0.436262,4.306890,8.666667,3.033333,23.381033
3,1.093634e+07,0.477704,4.667894,8.000000,4.088653,25.823495
4,4.774837e+07,0.557925,5.000000,8.666667,3.966667,20.357615


In [52]:
data[data.y_pred == 1].groupby('cluster').mean()

,Deposit,Lat,Lng,ClosestStationID,ClosestPSDistance,MeanPSDistance,PSCount,Take,PDistance,Officers1000,FFLCount,AvgRating,Population,CrimeRate1000,y_pred
cluster,,,,,,,,,,,,,,,
0,2.711152e+08,41.172373,-74.328706,831.003018,4.131650,5.968846,5.033156,1.355574e+05,0.295155,2.238139,7.849615,3.547937,4.387369e+06,28.690033,1.0
1,3.485215e+11,40.756200,-73.976400,896.000000,3.320694,3.320694,1.000000,1.742607e+08,0.367931,5.000000,10.000000,1.000000,8.566917e+06,20.357615,1.0
2,5.530666e+10,40.755800,-73.980550,896.000000,3.217315,3.217315,1.000000,2.765333e+07,0.378269,5.000000,9.500000,2.350000,8.566917e+06,20.357615,1.0
3,1.966710e+10,40.754575,-73.979300,896.000000,3.163166,3.163166,1.000000,9.833551e+06,0.383683,5.000000,9.750000,3.782979,8.566917e+06,20.357615,1.0
4,8.730121e+10,40.752100,-73.982300,896.000000,2.940012,2.940012,1.000000,4.365060e+07,0.405999,5.000000,10.000000,5.000000,8.566917e+06,20.357615,1.0


In [55]:
# Drop columns
dftarget2 = data[data.y_pred == 1].copy()
dftarget2.drop(['UniqueNum', 
       'Deposit', 'Lat', 'Lng', 'ClosestStationID',
       'MeanPSDistance', 'PSCount', 'Target', 'Population'], axis=1, inplace=True)

## Top 5 banks with highest Take

In [57]:
# Top 5 banks with highest Take and lower pDistance values from K-Means centers
dftop5 = dftarget2[dftarget2.cluster.isin([0, 1, 2])].sort_values(by=['Take'], ascending=[False]).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred,cluster
BankID,,,,,,,,,,,,,,,
8936,"JPMorgan Chase Bank, National Association",401 Madison Avenue,,New York,NY,10017,3.320694,174260736,0.367931,5.0,10,1.000000,20.357615,1,1
8934,"JPMorgan Chase Bank, National Association",401 Madison Avenue,,New York,NY,10017,3.320694,33282569,0.367931,5.0,10,1.000000,20.357615,1,2
8639,"Bank of America, National Association",115 West 42nd Street,,New York,NY,10036,3.113936,22024089,0.388606,5.0,9,3.700000,20.357615,1,2
8817,First Republic Bank,1230 Avenue Of The Americas,,New York,NY,10020,3.401485,4617530,0.359852,5.0,10,4.200000,20.357615,1,0
8684,"Capital One, National Association",57 West 57th Street,,New York,NY,10019,3.822906,4485512,0.317709,5.0,10,3.665959,20.357615,1,0


Not in this list:
- 8768 Citibank, National Association
- 8850 HSBC Bank USA, National Association


## Top 5 of banks with longest distance to Police Station

In [60]:
# Top 5 of banks with highest distance to Police Station
dftop5 = dftarget2[dftarget2.cluster.isin([0, 1, 2])].sort_values(by=['ClosestPSDistance'], ascending=False).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred,cluster
BankID,,,,,,,,,,,,,,,
7436,Berkshire Bank,West Main Street,,West Winfield,NY,13491,10.5,18044,0.0,0.0,2,3.665959,29.427869,1,0
9484,Greater Hudson Bank,485 Schutt Road Ext,,Middletown,NY,10940,10.5,27535,0.0,0.0,0,3.665959,29.439320,1,0
9417,"NBT Bank, National Association",Lyndon Corners Branch,,Syracuse,NY,13214,10.5,27081,0.0,0.0,0,3.665959,40.104221,1,0
10198,New York Community Bank,5770 Hylan Boulevard,,Staten Island,NY,10309,10.5,18347,0.0,0.0,1,3.665959,29.427869,1,0
10965,The First National Bank of Long Island,42 Deer Park Avenue,,Babylon,NY,11702,10.5,17092,0.0,0.0,9,1.000000,29.427869,1,0


## Sort by lowest officers rate per 1000

In [59]:
#Sort by lowest officers rate per 1000
dftop5 = dftarget2[dftarget2.cluster.isin([0, 1, 2])].sort_values(by=['Officers1000'], ascending=True).head(5)
dftop5

,Name,Address1,Address2,City,State,Zip,ClosestPSDistance,Take,PDistance,Officers1000,FFLCount,AvgRating,CrimeRate1000,y_pred,cluster
BankID,,,,,,,,,,,,,,,
11533,The Bank of Castile,1 Main Street,,Gainesville,NY,14066,6.481743,19696,0.051826,0.0,5,3.665959,29.427869,1,0
8174,"Capital One, National Association",195 Merritt Road,,Farmingdale,NY,11735,8.102861,84148,0.000000,0.0,16,3.665959,29.427869,1,0
9917,"JPMorgan Chase Bank, National Association",9059 Sutphin Boulevard,,Jamaica,NY,11435,5.560809,23801,0.143919,0.0,4,3.665959,29.427869,1,0
8184,"Capital One, National Association",3721 Hempstead Turnpike,,Levittown,NY,11756,6.607309,88312,0.039269,0.0,11,1.000000,29.427869,1,0
8188,"Capital One, National Association",4374 Sunrise Highway,,Massapequa,NY,11758,6.104679,45255,0.089532,0.0,13,3.665959,29.427869,1,0
